In [9]:
from langchain import hub


In [3]:
from langgraph.prebuilt import create_react_agent
from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(model="deepseek-chat", temperature=1.0)


def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model=llm,  
    tools=[get_weather],  
    prompt="You are a helpful assistant"  
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='6d6f72e5-4d1b-456e-95fd-fe7932abc08a'),
  AIMessage(content="I'll get the weather information for San Francisco for you.", additional_kwargs={'tool_calls': [{'id': 'call_00_i6EiCpsqVRQagDF5HF0sz7jh', 'function': {'arguments': '{"city": "San Francisco"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 158, 'total_tokens': 185, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 158}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_08f168e49b_prod0820_fp8_kvcache', 'id': '86117c32-ab37-4a5a-862a-52b8efbb3a60', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6848ace7-7777-4a92-b69d-41600fc79038-0', tool_calls=[{'name': 'get_w

In [ ]:
agent.get_graph().draw_mermaid_png()

# To save PNG to file:
png_data = agent.get_graph().draw_mermaid_png()
with open("graph.png", "wb") as f:
    f.write(png_data)

# # For terminal/ASCII output:
# agent.get_graph().draw_ascii()

ImportError: Install grandalf to draw graphs: `pip install grandalf`.

In [10]:
prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")


In [12]:
print(prompt_template)
type(prompt_template)

input_variables=['dialect', 'top_k'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'langchain-ai', 'lc_hub_repo': 'sql-agent-system-prompt', 'lc_hub_commit_hash': '31156d5fe3945188ee172151b086712d22b8c70f8f1c0505f5457594424ed352'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['dialect', 'top_k'], input_types={}, partial_variables={}, template='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\

langchain_core.prompts.chat.ChatPromptTemplate

In [13]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from typing import List, Optional

# 1. 定义单个天气对象的结构（Pydantic模型）
class DailyWeather(BaseModel):
    date: str = Field(description="日期，格式为YYYY-MM-DD")
    temperature: int = Field(description="温度，单位为摄氏度")
    condition: str = Field(description="天气状况，如'晴天'、'多云'、'小雨'")
    humidity: Optional[int] = Field(description="湿度，百分比，可选字段")  # 可选字段

# 2. 定义工具的输入参数结构（可选，用于参数校验）
class WeatherQueryInput(BaseModel):
    location: str = Field(description="城市和地区，例如：San Francisco, CA 或 北京市")

# 3. 定义工具，返回List[DailyWeather]（复杂出参）
@tool(args_schema=WeatherQueryInput)
def get_weather_forecast(location: str) -> List[DailyWeather]:
    """
    获取指定地点未来3天的天气预测，返回包含每日天气详情的列表。
    
    参数:
        location: 地点（城市+地区，例如"上海市"或"New York, NY"）
    
    返回结果说明:
        一个包含3个元素的列表，每个元素是一个天气对象，结构如下：
        - date: 日期（字符串，格式YYYY-MM-DD）
        - temperature: 温度（整数，单位℃）
        - condition: 天气状况（字符串，如"晴天"、"雷阵雨"）
        - humidity: 湿度（整数，百分比，可能为null）
    
    示例返回:
        [
            {"date": "2024-09-19", "temperature": 25, "condition": "晴天", "humidity": 60},
            {"date": "2024-09-20", "temperature": 23, "condition": "多云", "humidity": 65},
            {"date": "2024-09-21", "temperature": 20, "condition": "小雨", "humidity": 80}
        ]
    """
    # 模拟调用天气API获取数据（实际中替换为真实API调用）
    mock_data = [
        {"date": "2024-09-19", "temperature": 25, "condition": "晴天", "humidity": 60},
        {"date": "2024-09-20", "temperature": 23, "condition": "多云", "humidity": 65},
        {"date": "2024-09-21", "temperature": 20, "condition": "小雨", "humidity": 80}
    ]
    
    # 转换为Pydantic模型列表（确保返回格式严格符合定义）
    return [DailyWeather(**item) for item in mock_data]

In [14]:
print(get_weather_forecast.name)
print(get_weather_forecast.description)
print(get_weather_forecast.args)
print(get_weather_forecast.return_direct)

get_weather_forecast
获取指定地点未来3天的天气预测，返回包含每日天气详情的列表。

参数:
    location: 地点（城市+地区，例如"上海市"或"New York, NY"）

返回结果说明:
    一个包含3个元素的列表，每个元素是一个天气对象，结构如下：
    - date: 日期（字符串，格式YYYY-MM-DD）
    - temperature: 温度（整数，单位℃）
    - condition: 天气状况（字符串，如"晴天"、"雷阵雨"）
    - humidity: 湿度（整数，百分比，可能为null）

示例返回:
    [
        {"date": "2024-09-19", "temperature": 25, "condition": "晴天", "humidity": 60},
        {"date": "2024-09-20", "temperature": 23, "condition": "多云", "humidity": 65},
        {"date": "2024-09-21", "temperature": 20, "condition": "小雨", "humidity": 80}
    ]
{'location': {'description': '城市和地区，例如：San Francisco, CA 或 北京市', 'title': 'Location', 'type': 'string'}}
False


In [15]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client()
prompt = client.pull_prompt("takatost/conversation-title-generator", include_model=True)

In [22]:
type(prompt)

langchain_core.prompts.chat.ChatPromptTemplate

In [21]:
prompt.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'prompts', 'chat', 'ChatPromptTemplate'],
 'kwargs': {'input_variables': ['question'],
  'metadata': {'lc_hub_owner': 'takatost',
   'lc_hub_repo': 'conversation-title-generator',
   'lc_hub_commit_hash': '1e5cbaa4b1cb9a7a24a95123c99e76009b56e05f3441ffbd2b397e1918197e25'},
  'messages': [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You need to decompose the user\'s input into "subject" and "intention" in order to accurately figure out what the user\'s input language actually is. \nNotice: the language type user use could be diverse, which can be English, Chinese, Español, Arabic, Japanese, French, and etc.\nMAKE SURE your output is the SAME language as the user\'s input!\nYour output is restricted only to: (Input language) Intention + Subject(short as possible)\nYour output MUST be a valid JSON.\n\nTip: When the user\'s question is directed at you (the languag

In [23]:
from langchain.prompts import ChatPromptTemplate

# 定义消息列表：系统消息 + 用户消息
messages = [
    ("system", "你是一个{language}翻译助手，将输入文本翻译成{target_language}。"),
    ("user", "请翻译：{text}")
]

# 创建ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_messages(messages)

# 格式化提示词（传入变量）
formatted_prompt = prompt_template.format_prompt(
    language="中文",
    target_language="英文",
    text="我爱自然语言处理"
)

# 输出格式化后的消息列表
print(formatted_prompt.messages)

[SystemMessage(content='你是一个中文翻译助手，将输入文本翻译成英文。', additional_kwargs={}, response_metadata={}), HumanMessage(content='请翻译：我爱自然语言处理', additional_kwargs={}, response_metadata={})]


In [37]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from typing import Annotated, List, Optional, TypedDict
from langchain.schema import BaseMessage, HumanMessage, AIMessage

# 1. 定义系统状态结构 - 使用TypedDict确保字典形式访问
class ChatState(TypedDict):
    """对话系统的完整状态，使用字典形式访问"""
    # 对话消息列表，使用add_messages注解自动处理消息合并
    messages: Annotated[List[BaseMessage], add_messages]
    # 记忆组件，用于持久化存储对话历史
    memory: Optional[ConversationBufferMemory]
    # 当前用户输入
    user_input: str

# 2. 初始化记忆组件
def init_memory() -> ConversationBufferMemory:
    """初始化对话记忆组件"""
    return ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,  # 确保返回Message对象而非字符串
        max_token_limit=2000  # 限制记忆的最大token数，防止溢出
    )

# 3. 定义ChatPromptTemplate模板
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个友好且有帮助的对话助手。"
               "请根据对话历史 context 来理解用户当前的问题，并提供自然、连贯的回答。"
               "如果没有对话历史，直接回答当前问题即可。"),
    MessagesPlaceholder(variable_name="chat_history"),  # 来自记忆的对话历史
    ("user", "{user_input}")  # 当前用户输入
])

# 4. 初始化LLM模型 - 这里使用了正确的引用方式
llm = ChatDeepSeek(model="deepseek-chat", temperature=1.0)

# 5. 定义节点函数

def load_memory(state: ChatState) -> ChatState:
    """加载记忆到状态中"""
    if not state["memory"]:
        state["memory"] = init_memory()
    return state

def update_memory_with_user_input(state: ChatState) -> ChatState:
    """将用户输入更新到记忆中"""
    if state["memory"] and state["user_input"]:
        # 向记忆添加用户消息
        state["memory"].save_context(
            {"input": state["user_input"]},
            {"output": ""}  # 暂时为空，等待助手回复
        )
    return state

def generate_response(state: ChatState) -> ChatState:
    """生成助手回复"""
    if not state["memory"]:
        state["memory"] = init_memory()
    
    # 从记忆中获取对话历史
    chat_history = state["memory"].load_memory_variables({}).get("chat_history", [])
    
    # 使用ChatPromptTemplate格式化提示
    formatted_prompt = prompt_template.format_prompt(
        chat_history=chat_history,
        user_input=state["user_input"]
    )
    
    # 调用LLM生成回复
    response = llm.invoke(formatted_prompt.to_messages())
    
    # 更新状态消息列表
    # 创建新的消息列表以避免直接修改原始列表
    new_messages = state["messages"].copy()
    new_messages.append(HumanMessage(content=state["user_input"]))
    new_messages.append(AIMessage(content=response.content))
    
    # 将助手回复更新到记忆中
    state["memory"].save_context(
        {"input": state["user_input"]},
        {"output": response.content}
    )
    
    # 返回更新后的状态
    return {
        "messages": new_messages,
        "memory": state["memory"],
        "user_input": state["user_input"]
    }

# 6. 构建LangGraph图
def build_graph():
    """构建对话流程图"""
    graph_builder = StateGraph(ChatState)
    
    # 添加节点
    graph_builder.add_node("load_memory", load_memory)
    graph_builder.add_node("update_memory_with_user_input", update_memory_with_user_input)
    graph_builder.add_node("generate_response", generate_response)
    
    # 定义节点间的连接
    graph_builder.set_entry_point("load_memory")
    graph_builder.add_edge("load_memory", "update_memory_with_user_input")
    graph_builder.add_edge("update_memory_with_user_input", "generate_response")
    graph_builder.set_finish_point("generate_response")
    
    # 编译图
    return graph_builder.compile()

# 7. 运行对话系统
def run_chatbot():
    """运行多轮对话"""
    global graph
    graph = build_graph()
    # 初始化状态为字典形式
    current_state: ChatState = {
        "messages": [],
        "memory": None,
        "user_input": ""
    }
    
    print("对话助手已启动，输入 'exit' 退出对话\n")
    
    while True:
        user_input = input("你: ")
        if user_input.lower() in ["exit", "quit"]:
            print("助手: 再见！")
            break
        
        # 更新当前状态的用户输入
        current_state["user_input"] = user_input
        
        # 运行图获取更新后的状态
        current_state = graph.invoke(current_state)
        
        # 输出助手回复（最后一条消息）
        print(f"助手: {current_state["messages"][-1].content}\n")



In [38]:
run_chatbot()


对话助手已启动，输入 'exit' 退出对话

助手: 你好，Bob！很高兴认识你。有什么我可以帮助你的吗？

助手: 我是你的对话助手，随时为你提供帮助！如果你有任何问题或需要建议，尽管告诉我哦。 😊

助手: 你是Bob呀！刚刚你告诉我的～需要我帮你记住什么其他信息吗？ 😄



KeyboardInterrupt: 

In [44]:
formatted_prompt = prompt_template.format_prompt(
    chat_history=[("user","我是bob"),("ai", "hi bob, nice to meet you")],
    user_input="天气不错"
)

In [45]:
formatted_prompt.to_messages()

[SystemMessage(content='你是一个友好且有帮助的对话助手。请根据对话历史 context 来理解用户当前的问题，并提供自然、连贯的回答。如果没有对话历史，直接回答当前问题即可。', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='我是bob', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi bob, nice to meet you', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='天气不错', additional_kwargs={}, response_metadata={})]

In [48]:
from langchain.agents import initialize_agent, AgentType
from langchain_openai import ChatOpenAI
from langchain_community.tools.calculator import CalculatorTool  # 注意这里的路径变化

# 初始化大语言模型

# 加载计算器工具
tools = [CalculatorTool()]

# 初始化代理
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 测试计算
result = agent.run("3的平方加上(15除以3)的结果是多少？")
print(result)

ModuleNotFoundError: No module named 'langchain_community.tools.calculator'